In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from deepdrr.utils import test_utils
from deepdrr.two.renderer import *
from deepdrr.two.profiles import *

from deepdrr.two.renderable import *
from deepdrr.two.devices import *



In [3]:
f = test_utils.download_sampledata("CT-chest")
mf = Path("tests/resources/suzanne.stl")
mf2 = Path("tests/resources/threads.stl")
print(f)

Using downloaded and verified file: /home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd
/home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd


In [4]:
volume = Volume.from_nrrd(f)

mesh1 = Mesh.from_stl(mf, tags=["mesh1"])
mesh2 = Mesh.from_stl(mf2, tags=["mesh2"])

carm = MobileCArm()

graph = TransformTree()
graph.add(carm)
graph.add(volume)
graph.add(mesh1)
graph.add(mesh2)

scene = GraphScene(graph)

drr_json_path = Path("drr.json")
rast_json_path = Path("rasterize.json")

drr_settings = DRRRenderSettings(
    width=512,
    height=512,
    neglog=True,
)
drr_prof = DRRRenderProfile(
    settings=drr_settings,
    renderer=DeferredRenderer(drr_json_path),
    # renderer=SynchronousRenderer(),
    scene=scene
)

rast_settings = RasterizeRenderSettings(
    width=512,
    height=512,
    max_mesh_hits=32,
)
rasterize_prof = RasterizeRenderProfile(
    settings=rast_settings,
    renderer=DeferredRenderer(rast_json_path),
    # renderer=SynchronousRenderer(),
    scene=scene
)



In [5]:
drr_prof.__enter__()
rasterize_prof.__enter__()


In [6]:

mesh1.world_from_anatomical = geo.FrameTransform.from_translation([-30, 50, 200])


# with drr_prof, rasterize_prof:
for frame_idx in range(10):
    mesh2.world_from_anatomical = geo.FrameTransform.from_translation([0, 0, 10*frame_idx])

    carm.move_by(
        delta_isocenter=[0, 0, 10],
        delta_alpha=5*frame_idx,
        delta_beta=0,
        delta_gamma=0,
        degrees=False,
    )

    drr = drr_prof.render_drr(f"./drr_out/frame_{frame_idx:03d}.png")

    segs = rasterize_prof.render_seg(f"./seg_out/frame_{frame_idx:03d}.png", tags=["mesh1", "mesh2"])


    

In [7]:
drr_prof.__exit__(None, None, None)
rasterize_prof.__exit__(None, None, None)